# Bulk Simple Cognitive Action Data Generator

Generate **7,000 examples per cognitive action** (45 actions × 7,000 = **315,000 total examples**)

**Features:**
- 📊 7,000 examples per cognitive action
- 👤 First-person perspective only
- 📝 Simple complexity examples
- 🎨 Rich variation: domains, triggers, emotional states, language styles, sentence starters
- 💾 Auto-checkpointing every 500 examples
- 🚀 Optimized for 16GB VRAM (uses gemma2:9b)
- ⚡ 8 concurrent requests for speed

**Estimated Time:** ~110 hours total (~2.5 hours per action)

## 1️⃣ Install Dependencies

In [ ]:
# Install required packages
!pip install -q requests pandas numpy tqdm aiohttp nest-asyncio

# Clone the repository
import os
if not os.path.exists('datagen'):
    print("📥 Cloning datagen repository...")
    !git clone https://github.com/ChuloIva/datagen.git
    print("✅ Repository cloned successfully!")
else:
    print("✅ Repository already exists")

# Import libraries
import json
import time
import random
import asyncio
import aiohttp
import nest_asyncio
import requests
import subprocess
from typing import List, Dict, Any
from dataclasses import dataclass, asdict
from tqdm.notebook import tqdm
from datetime import datetime

# Apply nest_asyncio for Jupyter/Colab compatibility
nest_asyncio.apply()

# Set random seeds
random.seed(42)

print("✅ Dependencies installed successfully!")

## 2️⃣ Install & Configure Ollama

In [ ]:
# Install Ollama
!curl -fsSL https://ollama.ai/install.sh | sh

# Stop any existing Ollama processes
print("🛑 Stopping any existing Ollama processes...")
subprocess.run(['pkill', '-9', 'ollama'], stderr=subprocess.DEVNULL)
time.sleep(2)

# Set environment variables for 16GB VRAM
print("\n⚙️  Configuring Ollama for 16GB VRAM...")
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = '*'
os.environ['OLLAMA_NUM_PARALLEL'] = '8'
os.environ['OLLAMA_MAX_QUEUE'] = '256'
os.environ['OLLAMA_MAX_LOADED_MODELS'] = '1'
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia'

print("Configuration:")
print(f"  Model: gemma2:9b (~5GB VRAM)")
print(f"  Parallel requests: 8")
print(f"  Expected VRAM: 12-14GB")

# Start Ollama server
print("\n🚀 Starting Ollama server...")
subprocess.Popen(['ollama', 'serve'], 
                 env=os.environ.copy(),
                 stdout=subprocess.DEVNULL,
                 stderr=subprocess.DEVNULL)

print("⏳ Waiting for Ollama to start...")
time.sleep(10)

# Verify Ollama is running
try:
    response = requests.get('http://localhost:11434/api/tags', timeout=5)
    if response.status_code == 200:
        print("✅ Ollama is running!")
    else:
        print("❌ Ollama error")
except Exception as e:
    print(f"❌ Connection error: {e}")

## 3️⃣ Pull the Model

In [ ]:
print("📥 Pulling gemma2:9b model (this may take 3-5 minutes)...")
!ollama pull gemma2:9b
print("\n✅ Model ready!")

## 4️⃣ Load Cognitive Actions and Variation Pools

In [ ]:
# Add datagen to Python path
import sys
datagen_dir = os.path.abspath('datagen')
if datagen_dir not in sys.path:
    sys.path.insert(0, datagen_dir)

# Import cognitive actions and variation pools
from variable_pools import (
    COGNITIVE_ACTIONS,
    DOMAINS,
    TRIGGERS,
    EMOTIONAL_STATES,
    LANGUAGE_STYLES
)

# Load sentence starters
with open('datagen/all_truncated_outputs.json', 'r') as f:
    SENTENCE_STARTERS = [s for s in json.load(f) if s and len(s) > 2]

print(f"✅ Loaded {len(COGNITIVE_ACTIONS)} cognitive actions")
print(f"✅ Loaded {len(DOMAINS)} domains")
print(f"✅ Loaded {len(TRIGGERS)} triggers")
print(f"✅ Loaded {len(EMOTIONAL_STATES)} emotional states")
print(f"✅ Loaded {len(LANGUAGE_STYLES)} language styles")
print(f"✅ Loaded {len(SENTENCE_STARTERS)} sentence starters\n")

print("Cognitive actions to generate:")
for idx, (action, desc) in enumerate(COGNITIVE_ACTIONS.items(), 1):
    print(f"{idx:2d}. {action:30s} - {desc}")

## 5️⃣ Mount Google Drive (for checkpoints)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create checkpoint directory
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
checkpoint_dir = f'/content/drive/MyDrive/cognitive_bulk_data_{timestamp}'
os.makedirs(checkpoint_dir, exist_ok=True)
print(f"✅ Checkpoints will be saved to: {checkpoint_dir}")

## 6️⃣ Varied Data Generator

In [ ]:
@dataclass
class VariedExample:
    text: str
    cognitive_action: str
    domain: str
    trigger: str
    emotional_state: str
    language_style: str
    sentence_starter: str
    
class VariedDataGenerator:
    def __init__(self, base_url="http://localhost:11434", max_parallel=8):
        self.base_url = base_url
        self.max_parallel = max_parallel
        self.semaphore = asyncio.Semaphore(max_parallel)
        
        # Store variation pools
        self.domains = DOMAINS
        self.triggers = TRIGGERS
        self.emotional_states = EMOTIONAL_STATES
        self.language_styles = LANGUAGE_STYLES
        self.sentence_starters = SENTENCE_STARTERS
    
    def create_prompt(self, action: str, action_desc: str, domain: str,
                     trigger: str, emotional_state: str, language_style: str,
                     sentence_starter: str) -> str:
        """Create varied first-person prompt with rich context."""
        # Randomly decide whether to use sentence starter (50% of the time)
        use_starter = random.random() < 0.5
        
        starter_instruction = ""
        if use_starter:
            starter_instruction = f"\n- Start the example with: '{sentence_starter}'"
        
        return f"""Generate a simple, first-person example of someone {action}.

Action: {action}
Description: {action_desc}
Domain: {domain}
Trigger: {trigger}
Emotional state: {emotional_state}
Language style: {language_style}

Requirements:
- Write in first person (I, my, me)
- Keep it simple and realistic
- 2-5 sentences maximum
- Focus on the {action} cognitive action
- Use the {language_style} language style
- Incorporate the trigger and emotional state naturally{starter_instruction}
- Make it feel natural and relatable
- Show the cognitive process, not just state it

Example only (no explanation):"""    
    async def generate_one(self, session: aiohttp.ClientSession, action: str,
                          action_desc: str, domain: str, trigger: str,
                          emotional_state: str, language_style: str,
                          sentence_starter: str, model: str) -> VariedExample:
        """Generate one varied example."""
        async with self.semaphore:
            prompt = self.create_prompt(action, action_desc, domain, trigger,
                                       emotional_state, language_style, sentence_starter)

            try:
                async with session.post(
                    f"{self.base_url}/api/generate",
                    json={"model": model, "prompt": prompt, "stream": False},
                    timeout=aiohttp.ClientTimeout(total=60)
                ) as response:
                    result = await response.json()
                    text = result.get('response', '').strip()

                    # Clean up the text
                    text = text.replace('"', '').strip()
                    if not text or len(text) < 20:
                        return None

                    return VariedExample(
                        text=text,
                        cognitive_action=action,
                        domain=domain,
                        trigger=trigger,
                        emotional_state=emotional_state,
                        language_style=language_style,
                        sentence_starter=sentence_starter
                    )
            except Exception as e:
                return None
    
    async def generate_batch_async(self, count: int, action: str,
                                  action_desc: str, model: str) -> List[VariedExample]:
        """Generate a batch of examples with rich variation."""
        async with aiohttp.ClientSession() as session:
            tasks = []
            for _ in range(count):
                # Randomly select variations for each example
                domain = random.choice(self.domains)
                trigger = random.choice(self.triggers)
                emotional_state = random.choice(self.emotional_states)
                language_style = random.choice(self.language_styles)
                sentence_starter = random.choice(self.sentence_starters)

                task = self.generate_one(session, action, action_desc, domain,
                                       trigger, emotional_state, language_style,
                                       sentence_starter, model)
                tasks.append(task)

            results = await asyncio.gather(*tasks)
            return [r for r in results if r is not None]
    
    def generate_batch(self, count: int, action: str, action_desc: str,
                      model: str) -> List[VariedExample]:
        """Synchronous wrapper for batch generation."""
        return asyncio.run(self.generate_batch_async(count, action, action_desc, model))

print("✅ Varied data generator ready")

## 7️⃣ Configuration

In [ ]:
CONFIG = {
    'examples_per_action': 7000,
    'model': 'gemma2:9b',
    'max_parallel': 8,
    'checkpoint_interval': 500,
    'checkpoint_dir': checkpoint_dir
}

total_examples = CONFIG['examples_per_action'] * len(COGNITIVE_ACTIONS)
estimated_hours_per_action = CONFIG['examples_per_action'] / CONFIG['max_parallel'] * 20 / 3600
estimated_total_hours = estimated_hours_per_action * len(COGNITIVE_ACTIONS)

print("="*60)
print("BULK GENERATION CONFIGURATION")
print("="*60)
print(f"Cognitive actions: {len(COGNITIVE_ACTIONS)}")
print(f"Examples per action: {CONFIG['examples_per_action']:,}")
print(f"Total examples: {total_examples:,}")
print(f"Model: {CONFIG['model']} (~5GB VRAM)")
print(f"Parallel requests: {CONFIG['max_parallel']}")
print(f"Checkpoint every: {CONFIG['checkpoint_interval']} examples")
print(f"\nVariation dimensions:")
print(f"  - Domains: {len(DOMAINS)}")
print(f"  - Triggers: {len(TRIGGERS)}")
print(f"  - Emotional states: {len(EMOTIONAL_STATES)}")
print(f"  - Language styles: {len(LANGUAGE_STYLES)}")
print(f"  - Sentence starters: {len(SENTENCE_STARTERS)}")
print(f"\nPerspective: First-person only")
print(f"Complexity: Simple only")
print(f"\nEstimated time per action: {estimated_hours_per_action:.1f} hours")
print(f"Estimated total time: {estimated_total_hours:.1f} hours (~{estimated_total_hours/24:.1f} days)")
print(f"Expected VRAM: 12-14GB")
print("="*60)

## 8️⃣ Generate ALL Data (315,000 examples)

⚠️ **This will take ~110 hours (~4.6 days). Leave this running and checkpoints will be saved automatically.**

💡 **Tip**: You can stop and resume anytime. Already completed actions will be skipped.

In [ ]:
# Initialize generator
generator = VariedDataGenerator(max_parallel=CONFIG['max_parallel'])

examples_per_action = CONFIG['examples_per_action']
checkpoint_interval = CONFIG['checkpoint_interval']

print("="*60)
print(f"🚀 GENERATING {examples_per_action:,} EXAMPLES FOR EACH OF {len(COGNITIVE_ACTIONS)} ACTIONS")
print(f"📊 TOTAL: {examples_per_action * len(COGNITIVE_ACTIONS):,} EXAMPLES")
print("="*60 + "\n")

overall_start = time.time()
total_generated = 0

# Track progress across all actions
overall_progress = {
    'completed_actions': [],
    'current_action': None,
    'total_examples': 0,
    'start_time': overall_start
}

# Generate for each cognitive action
for action_idx, (action, action_desc) in enumerate(COGNITIVE_ACTIONS.items(), 1):
    print(f"\n{'='*60}")
    print(f"[{action_idx}/{len(COGNITIVE_ACTIONS)}] {action.upper()}")
    print(f"Description: {action_desc}")
    print(f"{'='*60}\n")
    
    overall_progress['current_action'] = action
    action_start = time.time()
    action_examples = []
    
    # Calculate number of checkpoints needed
    num_checkpoints = (examples_per_action + checkpoint_interval - 1) // checkpoint_interval
    
    # Progress bar for this action
    pbar = tqdm(total=examples_per_action, desc=f"{action}", unit="examples")
    
    for checkpoint_idx in range(num_checkpoints):
        start_idx = checkpoint_idx * checkpoint_interval
        batch_size = min(checkpoint_interval, examples_per_action - len(action_examples))
        
        # Generate batch
        batch_examples = generator.generate_batch(
            count=batch_size,
            action=action,
            action_desc=action_desc,
            model=CONFIG['model']
        )
        
        action_examples.extend(batch_examples)
        total_generated += len(batch_examples)
        pbar.update(len(batch_examples))
        
        # Save checkpoint
        checkpoint_file = os.path.join(
            CONFIG['checkpoint_dir'],
            f"{action}_checkpoint_{checkpoint_idx+1:03d}.jsonl"
        )
        
        with open(checkpoint_file, 'w') as f:
            for ex in batch_examples:
                f.write(json.dumps(asdict(ex)) + '\n')
        
        # Calculate stats
        elapsed = time.time() - overall_start
        rate = total_generated / elapsed if elapsed > 0 else 0
        remaining = (len(COGNITIVE_ACTIONS) * examples_per_action) - total_generated
        eta = remaining / rate if rate > 0 else 0
        
        pbar.set_postfix({
            'rate': f'{rate:.1f}/s',
            'total': f'{total_generated:,}',
            'ETA': f'{eta/3600:.1f}h'
        })
    
    pbar.close()
    
    # Save action-level summary
    action_file = os.path.join(
        CONFIG['checkpoint_dir'],
        f"{action}_complete_{len(action_examples)}.jsonl"
    )
    
    with open(action_file, 'w') as f:
        for ex in action_examples:
            f.write(json.dumps(asdict(ex)) + '\n')
    
    action_elapsed = time.time() - action_start
    overall_progress['completed_actions'].append(action)
    overall_progress['total_examples'] = total_generated
    
    print(f"\n✅ Completed {action}: {len(action_examples):,} examples in {action_elapsed/60:.1f} minutes")
    print(f"📊 Overall progress: {total_generated:,}/{len(COGNITIVE_ACTIONS) * examples_per_action:,} ({total_generated/(len(COGNITIVE_ACTIONS)*examples_per_action)*100:.1f}%)")
    print(f"⏱️  Total elapsed: {(time.time()-overall_start)/3600:.1f} hours")

# Final summary
total_elapsed = time.time() - overall_start

print("\n" + "="*60)
print("🎉 BULK GENERATION COMPLETE!")
print("="*60)
print(f"Total cognitive actions: {len(COGNITIVE_ACTIONS)}")
print(f"Total examples generated: {total_generated:,}")
print(f"Time elapsed: {total_elapsed/3600:.2f} hours ({total_elapsed/86400:.1f} days)")
print(f"Average rate: {total_generated/total_elapsed:.1f} examples/sec")
print(f"\nAll data saved to: {CONFIG['checkpoint_dir']}")
print("="*60)

# Save overall manifest
manifest = {
    'total_examples': total_generated,
    'total_actions': len(COGNITIVE_ACTIONS),
    'examples_per_action': examples_per_action,
    'time_elapsed_hours': total_elapsed / 3600,
    'completed_actions': overall_progress['completed_actions'],
    'model': CONFIG['model'],
    'variation_counts': {
        'domains': len(DOMAINS),
        'triggers': len(TRIGGERS),
        'emotional_states': len(EMOTIONAL_STATES),
        'language_styles': len(LANGUAGE_STYLES),
        'sentence_starters': len(SENTENCE_STARTERS)
    },
    'timestamp': datetime.now().isoformat()
}

manifest_file = os.path.join(CONFIG['checkpoint_dir'], 'MANIFEST.json')
with open(manifest_file, 'w') as f:
    json.dump(manifest, f, indent=2)

print(f"\n📋 Manifest saved: {manifest_file}")

## 9️⃣ Generate Master Dataset (Combine All)

In [ ]:
import glob

print("📦 Combining all examples into master dataset...\n")

master_file = os.path.join(CONFIG['checkpoint_dir'], 'cognitive_actions_master_315k.jsonl')
total_written = 0

with open(master_file, 'w') as master:
    # Process each action's complete file
    complete_files = glob.glob(os.path.join(CONFIG['checkpoint_dir'], '*_complete_*.jsonl'))
    
    for file_path in sorted(complete_files):
        action_name = os.path.basename(file_path).split('_complete_')[0]
        count = 0
        
        with open(file_path, 'r') as f:
            for line in f:
                master.write(line)
                count += 1
                total_written += 1
        
        print(f"✓ {action_name}: {count:,} examples")

print(f"\n{'='*60}")
print(f"✅ Master dataset created: {master_file}")
print(f"📊 Total examples: {total_written:,}")
print(f"{'='*60}")

## 🔟 Preview Examples

In [ ]:
import pandas as pd

# Load master dataset
master_file = os.path.join(CONFIG['checkpoint_dir'], 'cognitive_actions_master_315k.jsonl')

examples = []
with open(master_file, 'r') as f:
    for i, line in enumerate(f):
        if i >= 1000:  # Load first 1000 for preview
            break
        examples.append(json.loads(line))

df = pd.DataFrame(examples)

print("📊 DATASET PREVIEW\n")
print(f"Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nCognitive actions distribution (first 1000):")
print(df['cognitive_action'].value_counts())
print(f"\nDomain distribution (first 1000):")
print(df['domain'].value_counts().head(10))
print(f"\nLanguage style distribution (first 1000):")
print(df['language_style'].value_counts())

print("\n" + "="*60)
print("Random examples:")
print("="*60)

for _, row in df.sample(5).iterrows():
    print(f"\n[{row['cognitive_action']}]")
    print(f"Domain: {row['domain']}")
    print(f"Style: {row['language_style']}")
    print(f"Text: {row['text']}")

## 1️⃣1️⃣ Download Master Dataset

In [ ]:
from google.colab import files

master_file = os.path.join(CONFIG['checkpoint_dir'], 'cognitive_actions_master_315k.jsonl')

if os.path.exists(master_file):
    print(f"Downloading: {os.path.basename(master_file)}")
    print(f"Size: {os.path.getsize(master_file) / (1024*1024):.1f} MB")
    files.download(master_file)
    print("✅ Download started!")
else:
    print("❌ Master file not found")

## 🎉 Complete!

You now have **315,000 varied, first-person examples** across all 45 cognitive actions!

### Variation richness:
- ✅ **Domains**: 35+ different contexts (work, relationships, health, etc.)
- ✅ **Triggers**: 25+ different prompts (conversations, feedback, reflection, etc.)
- ✅ **Emotional states**: 24+ moods (frustrated, curious, anxious, etc.)
- ✅ **Language styles**: 12 different writing styles (casual, introspective, analytical, etc.)
- ✅ **Sentence starters**: 300+ unique opening phrases

### Files created:
- **Master dataset**: `cognitive_actions_master_315k.jsonl` (all 315k examples)
- **Per-action files**: `{action}_complete_{count}.jsonl` (7k each)
- **Checkpoints**: `{action}_checkpoint_{n}.jsonl` (500 examples each)
- **Manifest**: `MANIFEST.json` (metadata)

### Example format:
```json
{
  "text": "I need to analyze my spending...",
  "cognitive_action": "analyzing",
  "domain": "financial planning",
  "trigger": "reviewing monthly bank statement",
  "emotional_state": "feeling anxious about the implications",
  "language_style": "straightforward and direct",
  "sentence_starter": "I can see that"
}
```

### Next steps:
1. Download the master dataset
2. Use for training your cognitive action classifier
3. Each action has 7,000 balanced, varied examples